# Linear Customer Demand
- With Linear Customer Demand, We've to Determine how many wine bottles to sell during the wine period of 12 months
- Assumption: 1% of the Wine Bottles is sold by our Winery in Each Country 

In [1]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import math

In [2]:
# Initial demands for 2024 in millions
demands_2024 = {
    'Singapore': 16*0.01,
    'Thailand': 103*0.001,
    'Japan': 615*0.0001,
    'Korea': 814*0.0001,
    'Indonesia': 59*0.001
}

# Projected demands for 2033 in millions
demands_2033 = {
    'Singapore': 16 * 1.4 * 0.01,
    'Thailand': 103 * 1.5 * 0.001,
    'Japan': 615 * 1.3 * 0.0001,
    'Korea': 814 * 1.2 * 0.0001,
    'Indonesia': 59 * 1.4 * 0.001
}

# Define the range of years
years = list(range(2024, 2034))

# Calculate annual growth factors based on compounding annual growth rate formula
annual_growth_factors = {
    country: (demands_2033[country] / demands_2024[country]) ** (1 / (2033 - 2024)) 
    for country in demands_2024
}

# Prepare the DataFrame columns
columns = ['Month'] + list(demands_2024.keys())
data = []

# Loop over each year and country to calculate demands
for year in years:
    for month in range(1, 13):
        row = [datetime(year, month, 1).strftime('%b-%Y')]
        for country in demands_2024:
            i = years.index(year)
            base_demand = demands_2024[country] * (annual_growth_factors[country] ** i)
            seasonal_multiplier = 0.05 * np.sin((month / 12) * 2 * np.pi) + 1
            monthly_demand = (base_demand * seasonal_multiplier)/12
            row.append(monthly_demand)
        data.append(row)

# Create DataFrame
demand_df = pd.DataFrame(data, columns=columns)

# Prepare data for plotting directly from DataFrame
plot_data = []
for country in demands_2024.keys():
    plot_data.append(go.Bar(name=country, x=demand_df['Month'], y=demand_df[country]))

# Create the figure
fig = go.Figure(data=plot_data)

# Update the layout for a stacked bar chart
fig.update_layout(barmode='stack',
                  title='Projected Linear Wine Demand with Seasonal Fluctuations (2024-2034)',
                  xaxis_title='Year',
                  yaxis_title='Wine Demand (Millions)',
                  xaxis=dict(tickmode='array',
                             tickvals=demand_df['Month'][::12],  # Every January
                             ticktext=[date[-4:] for date in demand_df['Month'][::12]]))
fig.update_layout(template='plotly_dark')

# Show the figure
fig.show()


In [3]:
demand_df

,Month,Singapore,Thailand,Japan,Korea,Indonesia
0,Jan-2024,0.013667,0.008798,0.005253,0.006953,0.005040
1,Feb-2024,0.013911,0.008955,0.005347,0.007077,0.005130
2,Mar-2024,0.014000,0.009013,0.005381,0.007123,0.005163
3,Apr-2024,0.013911,0.008955,0.005347,0.007077,0.005130
4,May-2024,0.013667,0.008798,0.005253,0.006953,0.005040
...,...,...,...,...,...,...
115,Aug-2033,0.017858,0.012317,0.006374,0.007788,0.006585
116,Sep-2033,0.017733,0.012231,0.006329,0.007733,0.006539
117,Oct-2033,0.017858,0.012317,0.006374,0.007788,0.006585
118,Nov-2033,0.018200,0.012553,0.006496,0.007937,0.006711


In [4]:
demand_df.to_csv('../Output-Files/linear-Demand.csv', index=False)

# Simulation 1: Demand for all Cheap Wine is 1%
- From earlier Analysis 
    - Red Wine 1: Pinot Noir Ler Cru takes 18.53 months and is 125 euros per bottle
    - White Wine 1: Chardonnay Ler Cru takes 3.5 months and is 90 euros per bottle

- The First Time We Grape Harvest is in September 2024, followed by September 2025....
- Since the Demand is Constant, Find the Demand of wine for each Year and just sum it up 

In [5]:
sim1_demand_df = demand_df.copy()

# Create new columns for Red_Wine_1 and White_Wine_1 for each country and fill them with the divided values
sim1_demand_df['Singapore_Red_Wine_1'] = sim1_demand_df['Singapore'] * 0.5
sim1_demand_df['Singapore_White_Wine_1'] = sim1_demand_df['Singapore'] * 0.5

sim1_demand_df['Thailand_Red_Wine_1'] = sim1_demand_df['Thailand'] * 0.5
sim1_demand_df['Thailand_White_Wine_1'] = sim1_demand_df['Thailand'] * 0.5

sim1_demand_df['Japan_Red_Wine_1'] = sim1_demand_df['Japan'] * 0.5
sim1_demand_df['Japan_White_Wine_1'] = sim1_demand_df['Japan'] * 0.5

sim1_demand_df['Korea_Red_Wine_1'] = sim1_demand_df['Korea'] * 0.5
sim1_demand_df['Korea_White_Wine_1'] = sim1_demand_df['Korea'] * 0.5

sim1_demand_df['Indonesia_Red_Wine_1'] = sim1_demand_df['Indonesia'] * 0.5
sim1_demand_df['Indonesia_White_Wine_1'] = sim1_demand_df['Indonesia'] * 0.5

# Drop the original columns
sim1_demand_df.drop(columns=['Singapore', 'Thailand', 'Japan', 'Korea', 'Indonesia'], inplace=True)

In [6]:
# sim1_demand_df

In [7]:
Red_Wine_1_oak_time = 18.5
White_Wine_1_oak_time = 3.5

In [8]:
# Define the starting and ending dates
start_date = datetime(2024, 10, 1)
end_date = datetime(2033, 10, 1)

# Define the oak times for each type of wine
oak_times = {
    'Red_Wine_1': 18,
    'White_Wine_1': 3,
}

# Create an empty DataFrame to store the summed demand
columns = ['Batch']
countries = ['Singapore', 'Thailand', 'Japan', 'Korea', 'Indonesia']
for country in countries:
    for wine_type in oak_times.keys():
        columns.append(f'{country}_{wine_type}')

summed_demand = pd.DataFrame(columns=columns)

# Iterate over each month
current_date = start_date
data_to_concat = []
while current_date < end_date:
    month_demand = {'Batch': current_date.strftime('%b-%Y')}
    
    # Calculate demand for each wine type for each country
    for country in countries:
        for wine_type, oak_time in oak_times.items():
            # Calculate the starting and ending dates for demand calculation
            start_demand_date = current_date + timedelta(days=30*oak_time)
            end_demand_date = start_demand_date + timedelta(days=365)
            
            # Initialize demand for this wine type
            total_demand = 0
            
            # Iterate over each month within the 12-month demand period
            demand_date = start_demand_date
            for _ in range(12):
                # Convert demand_date to the format in sim2_demand_df
                demand_date_str = demand_date.strftime('%b-%Y')
                # Sum up the demand for this month
                total_demand += sim1_demand_df[sim1_demand_df['Month'] == demand_date_str][f'{country}_{wine_type}'].sum()
                # Move to the next month
                demand_date += timedelta(days=30)  # Assuming each month has 30 days
                
            # Add the total demand to the dictionary
            month_demand[f'{country}_{wine_type}'] = total_demand
    
    # Append the monthly demand to the DataFrame
    data_to_concat.append(pd.DataFrame(month_demand, index=[0]))
    
    # Move to the next year
    current_date += timedelta(days=365)

# Concatenate the monthly demand DataFrames
summed_demand_1 = pd.concat(data_to_concat, ignore_index=True)

# Print or use the summed_demand DataFrame
summed_demand_1

,Batch,Singapore_Red_Wine_1,Singapore_White_Wine_1,Thailand_Red_Wine_1,Thailand_White_Wine_1,Japan_Red_Wine_1,Japan_White_Wine_1,Korea_Red_Wine_1,Korea_White_Wine_1,Indonesia_Red_Wine_1,Indonesia_White_Wine_1
0,Oct-2024,0.086777,0.082794,0.056803,0.053675,0.032762,0.031584,0.042532,0.041463,0.031999,0.030530
1,Oct-2025,0.090083,0.085947,0.059421,0.056149,0.033731,0.032518,0.043403,0.042312,0.033218,0.031693
2,Oct-2026,0.093514,0.089221,0.062159,0.058736,0.034729,0.033480,0.044291,0.043178,0.034483,0.032900
3,Oct-2027,0.097077,0.092620,0.065024,0.061443,0.035757,0.034470,0.045197,0.044062,0.035797,0.034154
4,Sep-2028,0.100775,0.096148,0.068020,0.064274,0.036814,0.035490,0.046122,0.044963,0.037161,0.035455
5,Sep-2029,0.104613,0.099811,0.071155,0.067236,0.037903,0.036540,0.047066,0.045883,0.038576,0.036805
6,Sep-2030,0.108598,0.103613,0.074434,0.070335,0.039024,0.037621,0.048029,0.046822,0.040046,0.038207
7,Sep-2031,0.092696,0.107560,0.063935,0.073576,0.033085,0.038734,0.040422,0.047781,0.034182,0.039663
8,Sep-2032,0.000000,0.111658,0.000000,0.076966,0.000000,0.039879,0.000000,0.048758,0.000000,0.041174
9,Sep-2033,0.000000,0.009333,0.000000,0.006438,0.000000,0.003331,0.000000,0.004070,0.000000,0.003442


In [9]:
# Prices of wines
prices = {
    'Red_Wine_1': 125,
    'White_Wine_1': 90,
    'Red_Wine_2': 260,
    'White_Wine_2': 200
}

# Calculate profit
Sim1_Profit = (summed_demand_1['Singapore_Red_Wine_1'] + summed_demand_1['Thailand_Red_Wine_1'] + summed_demand_1['Japan_Red_Wine_1'] + summed_demand_1['Korea_Red_Wine_1'] + summed_demand_1['Indonesia_Red_Wine_1']) * prices['Red_Wine_1'] + \
              (summed_demand_1['Singapore_White_Wine_1'] + summed_demand_1['Thailand_White_Wine_1'] + summed_demand_1['Japan_White_Wine_1'] + summed_demand_1['Korea_White_Wine_1'] + summed_demand_1['Indonesia_White_Wine_1']) * prices['White_Wine_1'] 


# Sim1_Profit

In [10]:
print("Total Profit for Simulation 1, Linear Demand:", Sim1_Profit.sum())

Total Profit for Simulation 1, Linear Demand: 504.88170007646033


In [11]:
Red_Wine1_sum = summed_demand_1['Singapore_Red_Wine_1'] + summed_demand_1['Thailand_Red_Wine_1'] + summed_demand_1['Japan_Red_Wine_1'] + summed_demand_1['Korea_Red_Wine_1'] + summed_demand_1['Indonesia_Red_Wine_1'] 


White_Wine1_sum = summed_demand_1['Singapore_White_Wine_1'] + summed_demand_1['Thailand_White_Wine_1'] + summed_demand_1['Japan_White_Wine_1'] + summed_demand_1['Korea_White_Wine_1'] + summed_demand_1['Indonesia_White_Wine_1']


In [12]:
# Number of Oak Barells 
OakBarrel_RedWine1 = Red_Wine1_sum.apply(lambda x: math.ceil((x * 1000000) / 200))
OakBarrel_RedWine1

0    1255
1    1300
2    1346
3    1395
4    1445
5    1497
6    1551
7    1322
8       0
9       0
dtype: int64

In [13]:
# Number of Oak Barells 
OakBarrel_WhiteWine1 = White_Wine1_sum.apply(lambda x: math.ceil((x * 1000000) / 200))
OakBarrel_WhiteWine1

0    1201
1    1244
2    1288
3    1334
4    1382
5    1432
6    1483
7    1537
8    1593
9     134
dtype: int64

# Simulation 2: Demand for all Cheap Wine is 0.7% and Expensive Wine is 0.3%
- From earlier Analysis 
    - Red Wine 1: Pinot Noir Ler Cru takes 18.53 months and is 125 euros per bottle
    - Red Wine 2: Pinot Noir Grand Cru takes 27.53 months and is 260 euros per bottle
    - White Wine 1: Chardonnay Ler Cru takes 3.5 months and is 90 euros per bottle
    - White Wine 2: Chardonnay Grand Cru takes 15.5 months and is 200 euros per bottle

- The First Time We Grape Harvest is in September 2024, followed by September 2025....
- Since the Demand is Constant, Find the Demand of wine for each Year and just sum it up 

In [14]:
sim2_demand_df = demand_df.copy()

# Create new columns for Red_Wine_1 and White_Wine_1 for each country and fill them with the divided values
sim2_demand_df['Singapore_Red_Wine_1'] = sim2_demand_df['Singapore'] * 0.35
sim2_demand_df['Singapore_White_Wine_1'] = sim2_demand_df['Singapore'] * 0.35
sim2_demand_df['Singapore_Red_Wine_2'] = sim2_demand_df['Singapore'] * 0.15
sim2_demand_df['Singapore_White_Wine_2'] = sim2_demand_df['Singapore'] * 0.15

sim2_demand_df['Thailand_Red_Wine_1'] = sim2_demand_df['Thailand'] * 0.35
sim2_demand_df['Thailand_White_Wine_1'] = sim2_demand_df['Thailand'] * 0.35
sim2_demand_df['Thailand_Red_Wine_2'] = sim2_demand_df['Thailand'] * 0.15
sim2_demand_df['Thailand_White_Wine_2'] = sim2_demand_df['Thailand'] * 0.15

sim2_demand_df['Japan_Red_Wine_1'] = sim2_demand_df['Japan'] * 0.35
sim2_demand_df['Japan_White_Wine_1'] = sim2_demand_df['Japan'] * 0.35
sim2_demand_df['Japan_Red_Wine_2'] = sim2_demand_df['Japan'] * 0.15
sim2_demand_df['Japan_White_Wine_2'] = sim2_demand_df['Japan'] * 0.15

sim2_demand_df['Korea_Red_Wine_1'] = sim2_demand_df['Korea'] * 0.35
sim2_demand_df['Korea_White_Wine_1'] = sim2_demand_df['Korea'] * 0.35
sim2_demand_df['Korea_Red_Wine_2'] = sim2_demand_df['Korea'] * 0.15
sim2_demand_df['Korea_White_Wine_2'] = sim2_demand_df['Korea'] * 0.15

sim2_demand_df['Indonesia_Red_Wine_1'] = sim2_demand_df['Indonesia'] * 0.35
sim2_demand_df['Indonesia_White_Wine_1'] = sim2_demand_df['Indonesia'] * 0.35
sim2_demand_df['Indonesia_Red_Wine_2'] = sim2_demand_df['Indonesia'] * 0.15
sim2_demand_df['Indonesia_White_Wine_2'] = sim2_demand_df['Indonesia'] * 0.15

# Drop the original columns
sim2_demand_df.drop(columns=['Singapore', 'Thailand', 'Japan', 'Korea', 'Indonesia'], inplace=True)

In [15]:
# Define the starting and ending dates
start_date = datetime(2024, 10, 1)
end_date = datetime(2033, 10, 1)

# Define the oak times for each type of wine
oak_times = {
    'Red_Wine_1': 18,
    'White_Wine_1': 3,
    'Red_Wine_2': 27,
    'White_Wine_2': 15
}

# Create an empty DataFrame to store the summed demand
columns = ['Batch']
countries = ['Singapore', 'Thailand', 'Japan', 'Korea', 'Indonesia']
for country in countries:
    for wine_type in oak_times.keys():
        columns.append(f'{country}_{wine_type}')

summed_demand = pd.DataFrame(columns=columns)

# Iterate over each month
current_date = start_date
data_to_concat = []
while current_date < end_date:
    month_demand = {'Batch': current_date.strftime('%b-%Y')}
    
    # Calculate demand for each wine type for each country
    for country in countries:
        for wine_type, oak_time in oak_times.items():
            # Calculate the starting and ending dates for demand calculation
            start_demand_date = current_date + timedelta(days=30*oak_time)
            end_demand_date = start_demand_date + timedelta(days=365)
            
            # Initialize demand for this wine type
            total_demand = 0
            
            # Iterate over each month within the 12-month demand period
            demand_date = start_demand_date
            for _ in range(12):
                # Convert demand_date to the format in sim2_demand_df
                demand_date_str = demand_date.strftime('%b-%Y')
                # Sum up the demand for this month
                total_demand += sim2_demand_df[sim2_demand_df['Month'] == demand_date_str][f'{country}_{wine_type}'].sum()
                # Move to the next month
                demand_date += timedelta(days=30)  # Assuming each month has 30 days
                
            # Add the total demand to the dictionary
            month_demand[f'{country}_{wine_type}'] = total_demand
    
    # Append the monthly demand to the DataFrame
    data_to_concat.append(pd.DataFrame(month_demand, index=[0]))
    
    # Move to the next year
    current_date += timedelta(days=365)

# Concatenate the monthly demand DataFrames
summed_demand_2 = pd.concat(data_to_concat, ignore_index=True)

# Print or use the summed_demand DataFrame
summed_demand_2

,Batch,Singapore_Red_Wine_1,Singapore_White_Wine_1,Singapore_Red_Wine_2,Singapore_White_Wine_2,Thailand_Red_Wine_1,Thailand_White_Wine_1,Thailand_Red_Wine_2,Thailand_White_Wine_2,Japan_Red_Wine_1,...,Japan_Red_Wine_2,Japan_White_Wine_2,Korea_Red_Wine_1,Korea_White_Wine_1,Korea_Red_Wine_2,Korea_White_Wine_2,Indonesia_Red_Wine_1,Indonesia_White_Wine_1,Indonesia_Red_Wine_2,Indonesia_White_Wine_2
0,Oct-2024,0.060744,0.057955,0.026766,0.025784,0.039762,0.037573,0.017621,0.016845,0.022934,...,0.010044,0.009755,0.029773,0.029024,0.012953,0.012694,0.022399,0.021371,0.009870,0.009508
1,Oct-2025,0.063058,0.060163,0.027786,0.026766,0.041595,0.039304,0.018433,0.017621,0.023612,...,0.010341,0.010044,0.030382,0.029618,0.013218,0.012953,0.023253,0.022185,0.010246,0.009870
2,Oct-2026,0.065460,0.062455,0.028845,0.027786,0.043511,0.041115,0.019282,0.018433,0.024310,...,0.010647,0.010341,0.031004,0.030225,0.013489,0.013218,0.024138,0.023030,0.010636,0.010246
3,Oct-2027,0.067954,0.064834,0.029943,0.028845,0.045517,0.043010,0.020171,0.019282,0.025030,...,0.010962,0.010647,0.031638,0.030843,0.013765,0.013489,0.025058,0.023908,0.011042,0.010636
4,Sep-2028,0.070542,0.067304,0.031084,0.029943,0.047614,0.044992,0.021100,0.020171,0.025770,...,0.011286,0.010962,0.032286,0.031474,0.014047,0.013765,0.026012,0.024818,0.011462,0.011042
5,Sep-2029,0.073229,0.069868,0.032268,0.031084,0.049808,0.047065,0.022073,0.021100,0.026532,...,0.011620,0.011286,0.032946,0.032118,0.014334,0.014047,0.027003,0.025764,0.011899,0.011462
6,Sep-2030,0.076019,0.072529,0.033497,0.032268,0.052103,0.049234,0.023090,0.022073,0.027317,...,0.011964,0.011620,0.033621,0.032776,0.014628,0.014334,0.028032,0.026745,0.012352,0.011899
7,Sep-2031,0.064887,0.075292,0.002800,0.033497,0.044755,0.051503,0.001931,0.023090,0.023159,...,0.000999,0.011964,0.028295,0.033446,0.001221,0.014628,0.023927,0.027764,0.001033,0.012352
8,Sep-2032,0.000000,0.078160,0.000000,0.002800,0.000000,0.053876,0.000000,0.001931,0.000000,...,0.000000,0.000999,0.000000,0.034131,0.000000,0.001221,0.000000,0.028822,0.000000,0.001033
9,Sep-2033,0.000000,0.006533,0.000000,0.000000,0.000000,0.004506,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.002849,0.000000,0.000000,0.000000,0.002409,0.000000,0.000000


In [16]:
# Prices of wines
prices = {
    'Red_Wine_1': 125,
    'White_Wine_1': 90,
    'Red_Wine_2': 260,
    'White_Wine_2': 200
}

# Calculate profit
Sim2_Profit = (summed_demand_2['Singapore_Red_Wine_1'] + summed_demand_2['Thailand_Red_Wine_1'] + summed_demand_2['Japan_Red_Wine_1'] + summed_demand_2['Korea_Red_Wine_1'] + summed_demand_2['Indonesia_Red_Wine_1']) * prices['Red_Wine_1'] + \
              (summed_demand_2['Singapore_White_Wine_1'] + summed_demand_2['Thailand_White_Wine_1'] + summed_demand_2['Japan_White_Wine_1'] + summed_demand_2['Korea_White_Wine_1'] + summed_demand_2['Indonesia_White_Wine_1']) * prices['White_Wine_1'] + \
              (summed_demand_2['Singapore_Red_Wine_2'] + summed_demand_2['Thailand_Red_Wine_2'] + summed_demand_2['Japan_Red_Wine_2'] + summed_demand_2['Korea_Red_Wine_2'] + summed_demand_2['Indonesia_Red_Wine_2']) * prices['Red_Wine_2'] + \
              (summed_demand_2['Singapore_White_Wine_2'] + summed_demand_2['Thailand_White_Wine_2'] + summed_demand_2['Japan_White_Wine_2'] + summed_demand_2['Korea_White_Wine_2'] + summed_demand_2['Indonesia_White_Wine_2']) * prices['White_Wine_2']

# Sim2_Profit

In [17]:
print("Total Profit for Simulation 2, Linear Demand:", Sim2_Profit.sum())

Total Profit for Simulation 2, Linear Demand: 649.2791912104714


In [18]:
Red_Wine1_sum = summed_demand_2['Singapore_Red_Wine_1'] + summed_demand_2['Thailand_Red_Wine_1'] + summed_demand_2['Japan_Red_Wine_1'] + summed_demand_2['Korea_Red_Wine_1'] + summed_demand_2['Indonesia_Red_Wine_1'] 

Red_Wine2_sum = summed_demand_2['Singapore_Red_Wine_2'] + summed_demand_2['Thailand_Red_Wine_2'] + summed_demand_2['Japan_Red_Wine_2'] + summed_demand_2['Korea_Red_Wine_2'] + summed_demand_2['Indonesia_Red_Wine_2'] 

White_Wine1_sum = summed_demand_2['Singapore_White_Wine_1'] + summed_demand_2['Thailand_White_Wine_1'] + summed_demand_2['Japan_White_Wine_1'] + summed_demand_2['Korea_White_Wine_1'] + summed_demand_2['Indonesia_White_Wine_1'] 

White_Wine2_sum = summed_demand_2['Singapore_White_Wine_2'] + summed_demand_2['Thailand_White_Wine_2'] + summed_demand_2['Japan_White_Wine_2'] + summed_demand_2['Korea_White_Wine_2'] + summed_demand_2['Indonesia_White_Wine_2']


In [19]:
# Number of Oak Barells 
OakBarrel_RedWine1 = Red_Wine1_sum.apply(lambda x: math.ceil((x * 1000000) / 200))
OakBarrel_RedWine1

0     879
1     910
2     943
3     976
4    1012
5    1048
6    1086
7     926
8       0
9       0
dtype: int64

In [20]:
# Number of Oak Barells 
OakBarrel_RedWine2 = Red_Wine2_sum.apply(lambda x: math.ceil((x * 1000000) / 200))
OakBarrel_RedWine2

0    387
1    401
2    415
3    430
4    445
5    461
6    478
7     40
8      0
9      0
dtype: int64

In [21]:
# Number of Oak Barells 
OakBarrel_WhiteWine1 = White_Wine1_sum.apply(lambda x: math.ceil((x * 1000000) / 200))
OakBarrel_WhiteWine1

0     841
1     871
2     902
3     934
4     968
5    1002
6    1039
7    1076
8    1115
9      94
dtype: int64

In [22]:
# Number of Oak Barells 
OakBarrel_WhiteWine2 = White_Wine2_sum.apply(lambda x: math.ceil((x * 1000000) / 200))
OakBarrel_WhiteWine2

0    373
1    387
2    401
3    415
4    430
5    445
6    461
7    478
8     40
9      0
dtype: int64